In [ ]:
import pandas as pd
from config import (
    LATITUDE_UPPER_BOUND,
    LATITUDE_LOWER_BOUND,
    LONGITUDE_UPPER_BOUND,
    LONGITUDE_LOWER_BOUND,
)

df = pd.read_csv("../data/Order/original/yellow_tripdata_2016-06.csv")

In [ ]:
df.columns

In [ ]:
USE_COLUMNS = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
]
RENAME_DICT = {
    "tpep_pickup_datetime": "Start_time",
    "tpep_dropoff_datetime": "End_time",
    "pickup_longitude": "PointS_Longitude",
    "pickup_latitude": "PointS_Latitude",
    "dropoff_longitude": "PointE_Longitude",
    "dropoff_latitude": "PointE_Latitude",
}


In [ ]:
len(df)

In [ ]:
rename_df = df[USE_COLUMNS].rename(columns=RENAME_DICT).sort_values("Start_time").reset_index(drop=True)

# NewYorkエリア外のレコードを除去する.
rename_without_outlier_df = rename_df[
    (rename_df["PointS_Longitude"] > LONGITUDE_LOWER_BOUND)
    & (rename_df["PointS_Longitude"] < LONGITUDE_UPPER_BOUND)
    & (rename_df["PointE_Longitude"] > LONGITUDE_LOWER_BOUND)
    & (rename_df["PointE_Longitude"] < LONGITUDE_UPPER_BOUND)
    & (rename_df["PointS_Latitude"] > LATITUDE_LOWER_BOUND)
    & (rename_df["PointS_Latitude"] < LATITUDE_UPPER_BOUND)
    & (rename_df["PointE_Latitude"] > LATITUDE_LOWER_BOUND)
    & (rename_df["PointE_Latitude"] < LATITUDE_UPPER_BOUND)
]

In [ ]:
from datetime import datetime, timedelta

rename_without_outlier_df["Start_time"] = rename_without_outlier_df["Start_time"].apply(
    lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
)

In [ ]:
start_date = datetime(year=2016, month=6, day=1)
while True:
    next_date = start_date + timedelta(days=1)
    tmp_df = rename_without_outlier_df[
        (rename_without_outlier_df["Start_time"]<next_date)
        & (rename_without_outlier_df["Start_time"]>start_date)
    ]
    tmp_df.to_csv(
        f"../data/Order/modified/order_2016{start_date.month}{str(start_date.day).zfill(2)}.csv",
        index=False
    )
    start_date = next_date
    if start_date.month != 6:
        break